### 1. Preprocessing

In [1]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from preprocessing import combine_data_frompath
import datetime
from tensorflow.keras import callbacks



run this code below to see that you are using GPU for computing

In [ ]:
tf.config.list_physical_devices("GPU")

In [2]:
train_data = combine_data_frompath('aug_data\\train\\labels\\*.json','aug_data\\train\\images\\*.jpg')
val_data = combine_data_frompath('aug_data\\val\\labels\\*.json','aug_data\\val\\images\\*.jpg')

### 2. Create Model

In [3]:
from model import FaceTracker
facetracker = FaceTracker()

In [4]:
batches_per_epoch = len(train_data)
lr_decay = (1./0.75 -1)/batches_per_epoch
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)
facetracker.compile(optimizer=opt)

### 3. Train Model

## 3.1 Create Callback

In [5]:
# this is for the tensorboard we will plot the training loss in tensorboard
time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/training/" + time 
tf_board_callback = callbacks.TensorBoard(log_dir=logdir)
early_stopping_callback = callbacks.EarlyStopping(monitor="val_total_loss",patience=3)
num_epochs = 10

## 3.2 Run Training 

In [6]:
facetracker.fit(train_data,epochs=num_epochs,
                callbacks=[
                            tf_board_callback,
                            early_stopping_callback],
                validation_data=val_data,
                verbose=1)

Epoch 1/10
37/37 [==============================] - 26s 475ms/step - total_loss: 2.5377 - classloss: 0.3023 - coordloss: 2.2355 - val_total_loss: 0.2248 - val_classloss: 0.0185 - val_coordloss: 0.2063
Epoch 2/10
37/37 [==============================] - 14s 392ms/step - total_loss: 0.3482 - classloss: 0.0103 - coordloss: 0.3379 - val_total_loss: 0.0291 - val_classloss: 9.0777e-05 - val_coordloss: 0.0290
Epoch 3/10
37/37 [==============================] - 14s 392ms/step - total_loss: 0.0618 - classloss: 2.8780e-04 - coordloss: 0.0615 - val_total_loss: 0.0107 - val_classloss: 2.8360e-05 - val_coordloss: 0.0107
Epoch 4/10
37/37 [==============================] - 14s 391ms/step - total_loss: 0.0361 - classloss: 1.6228e-04 - coordloss: 0.0359 - val_total_loss: 0.0100 - val_classloss: 1.6306e-05 - val_coordloss: 0.0100
Epoch 5/10
37/37 [==============================] - 14s 389ms/step - total_loss: 0.0291 - classloss: 9.7596e-05 - coordloss: 0.0290 - val_total_loss: 0.0078 - val_classloss: 1.

### 4. Test Model

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()